In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
train_dataset = pd.read_csv('data/train.csv')
test_dataset = pd.read_csv('data/test.csv')

### Data preprocessing & Feature Engineering
Its always a good practice to go through the following checklists to preprocess our dataset
- Split the dataset into X (attributes) and y (label)
- Check if contains missing values
- Check if continuous attribute(s) contain any outliers
- Some misc properties of the dataset

In [3]:
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_dataset.info()
print('_')
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
_
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null flo

From the info object information, we can see that indeed the dataset contains missing values at the following column(s):
- Age
- Cabin

We will:
- apply **Imputer** with mean strategy to handle missing Age
- drop Cabin column since it is intuitively not useful in our analysis (as well as modeling)

In [94]:
X_train = train_dataset[[
    'Sex',
    'Pclass', 
    'Age', 
    'SibSp',
    'Parch',
    'Fare'
]].values
y_train = train_dataset[['Survived']].values

mean_fare = train_dataset.Fare.mean()
X_test = test_dataset[[
    'Sex',
    'Pclass',
    'Age',
    'SibSp',
    'Parch',
    'Fare'
]]
X_test['Fare'] = X_test['Fare'].fillna(mean_fare)
X_test = X_test.values

/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [95]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
train_imputer_col = imputer.fit_transform(X_train[:, 2:3])
test_imputer_col = imputer.transform(X_test[:, 2:3])
X_train[:, 2:3] = train_imputer_col
X_test[:, 2:3] = test_imputer_col

In [96]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

sex_encoder = LabelEncoder()

X_train[:, 0] = sex_encoder.fit_transform(X_train[:, 0])
X_test[:, 0] = sex_encoder.transform(X_test[:, 0])

onehotencoder = OneHotEncoder(categorical_features = [0])
X_train = onehotencoder.fit_transform(X_train).toarray()
X_test = onehotencoder.transform(X_test).toarray()

# # Avoid dummy variables trap
X_train = X_train[:, 1:]
X_test = X_test[:, 1:]

/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [98]:
# Prior to feature scaling
print(X_train[0])

# Apply feature scaling (a must if using neural network)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

[ 1.    3.   22.    1.    0.    7.25]


In [99]:
# After feature scaling
X_train[0]

array([ 0.73769513,  0.82737724, -0.5924806 ,  0.43279337, -0.47367361,
       -0.50244517])

### Training model with artificial neural network (ANN), with automatic feature engineering (using library)

In [100]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/Users/yimkh/python/bin/anaconda3/envs/kaggle-titanic/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [101]:
classifier = Sequential()
classifier.add(Dense(units=8, input_shape=(6,), activation='relu', kernel_initializer='uniform'))
classifier.add(Dense(units=16, activation='relu', kernel_initializer='uniform'))
classifier.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [102]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 56        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 217
Trainable params: 217
Non-trainable params: 0
_________________________________________________________________


In [103]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
891/891 [==============================] - 1s 709us/step - loss: 0.6864 - acc: 0.6207
Epoch 2/100
891/891 [==============================] - 0s 342us/step - loss: 0.5962 - acc: 0.7497
Epoch 3/100
891/891 [==============================] - 0s 233us/step - loss: 0.4820 - acc: 0.8070
Epoch 4/100
891/891 [==============================] - 0s 223us/step - loss: 0.4523 - acc: 0.8013
Epoch 5/100
891/891 [==============================] - 0s 297us/step - loss: 0.4411 - acc: 0.8092
Epoch 6/100
891/891 [==============================] - 0s 227us/step - loss: 0.4353 - acc: 0.8103
Epoch 7/100
891/891 [==============================] - 0s 214us/step - loss: 0.4325 - acc: 0.8081
Epoch 8/100
891/891 [==============================] - 0s 220us/step - loss: 0.4285 - acc: 0.8114
Epoch 9/100
891/891 [==============================] - 0s 230us/step - loss: 0.4264 - acc: 0.8103
Epoch 10/100
891/891 [==============================] - 0s 368us/step - loss: 0.4245 - acc: 0.8159
Epoch 11/100
891/89

891/891 [==============================] - 0s 243us/step - loss: 0.4019 - acc: 0.8361
Epoch 84/100
891/891 [==============================] - 0s 243us/step - loss: 0.4001 - acc: 0.8361
Epoch 85/100
891/891 [==============================] - 0s 271us/step - loss: 0.3987 - acc: 0.8350
Epoch 86/100
891/891 [==============================] - 0s 266us/step - loss: 0.3995 - acc: 0.8373
Epoch 87/100
891/891 [==============================] - 0s 257us/step - loss: 0.3987 - acc: 0.8395
Epoch 88/100
891/891 [==============================] - 0s 264us/step - loss: 0.3984 - acc: 0.8305
Epoch 89/100
891/891 [==============================] - 0s 239us/step - loss: 0.4000 - acc: 0.8395
Epoch 90/100
891/891 [==============================] - 0s 256us/step - loss: 0.3979 - acc: 0.8406
Epoch 91/100
891/891 [==============================] - 0s 282us/step - loss: 0.4004 - acc: 0.8361
Epoch 92/100
891/891 [==============================] - 0s 288us/step - loss: 0.3989 - acc: 0.8339
Epoch 93/100
891/891 [=

### Validate the accuracy on testing dataset